## Surprisal Models
**Brief**:<br>
This will be the main file for all of our model loading, data organizing, searching, etc.<br><br>
**Sections**:
1. [Methodology And Instructions For Running](#pre)
2. [Starting A StanfordCoreNLP Server](#1)
    - [Background](#1_a)
    - [How To Run The Server](#1_b)
    - [Resources](#1_c)
    - [Code](#1_d)
3. [Applying Models](#2)
___
<a id='pre'>

## Methodology And Instructions For Running
In this section, we'll summarize and catch up to where we are now.<br>
**Installations**:
There are two quick installations that will catch you up with the workflow of using Python.
1. Git Bash (https://git-scm.com/downloads)
    - This is a modified version of the command line (or terminal in MacOS) that enables the user to download GitHub repositories to their local machine, contribute to these repositories, and then publish their results.
    - I'm not going to do a big whole thing on Git here, but I will have you:
        - **clone** our repository to your machine, 
        - **add** a few files, 
        - **commit** your changes, 
        - and finally submit a **pull request**.
    - Why go through this hassle? Now, you can work with our code on your machine in Juypter, as opposed to having look at it on the GitHub interface!
2. Anaconda (https://www.anaconda.com/distribution/)
    - This is an extremely popular platform for data science work. I think you might already have it, so I won't go into much detail. Basically we'll be using this so we can access Juypter Notebooks (like this one!).
<br><br>

**A Road Map to Tag and Word Level Probability**:
It's a long road from a raw sentence to a number than represents the likelihood of a given word or tag. Let's see how we can get there.
1. Sentence Tagging
    - sentences -> list of tuples, each tuple has word and tag
    - sometimes this step will be consumed by the subsequent parsing step. for example, the Stanford parser accepts raw sentences as input, does the tagging, and then does the parsing.
        - *parser.raw_parse('The King of France is Bald.')* does this
2. Sentence Parsing
    - list of tags -> a tree
    - where we are at. need to talk to Na Rae

<a id='#1'>

### Starting A StanfordCoreNLP Server
<a id='1_a'>

**Background**:<br>
We've spent a lot of time looking at the StanfordCoreNLP software, and we ultimately decided we want to use both the parser and tagger (really, the parser automatically implements the tagger, but we'll see later). While the software is very useful, it's written in Java--which is not quite as nice to play with as Python for a number of reasons (lacks NLP libraries, lower level language, etc). The problem then is finding a way to use a Java program like it's a Python program.<br><br>
The two main options I looked into was a traditional import and a private server. One route of solving our problem is to use a traditional "wrapper" library/program. This program is essentially a translator between Java and Python. Unfortunately, the Stanford team itself doesn't actually make these wrappers (they would have to make them for a *lot* of languages). The existing wrappers--specifically the <u>stanfordcorenlp</u> library--weren't available through Anaconda (the platform through which we are running this exciting program right now), so I went another route.<br><br>
The direction I chose was to host a server that runs the out-of-the-box Java program, and to access it through a Python API. This involves a small amount of command line setup, but it saves the trouble of changing environment variables or using directories in Python.<br><br>
<a id='1_b'>

**How To Run A Server**<br>
*Note*: Huge thanks to Khalid Alnajjar, linked his guide in resources.
I've never actually hosted any sort of server before, so here's a quick summary:
1. download and extract the CoreNLP somewhere.
2. on the command line, cd into that directory
    - *stanford-corenlp-full-2018-10-05* should be the folder
3. run this command to host the server:
    - java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "tokenize,ssplit,pos,lemma,parse,sentiment" -port 9000 -timeout 30000 
4. pick up [here](#1_d)
<a id='1_c'>

**Resources**:<br>
1. documentation for the parser: http://www.nltk.org/_modules/nltk/parse/stanford.html
2. for more on running the server: https://www.khalidalnajjar.com/setup-use-stanford-corenlp-server-python/
3. StanfordCoreNLP's GitHub page: https://github.com/stanfordnlp/CoreNLP
4. StanfordCoreNLP's download (3.9.2): https://stanfordnlp.github.io/CoreNLP/
5. stanfordcorenlp wrapper library: https://pypi.org/project/stanfordcorenlp/

<a id='1_d'>

**Code**:

In [1]:
#Imports
import nltk
import pandas as pd
from stanfordcorenlp import StanfordCoreNLP
from nltk import StanfordPOSTagger
from nltk.parse import stanford
from nltk.parse import CoreNLPParser
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#builds parser
parser = CoreNLPParser(url='http://localhost:9000')

*Note*: How to make sense of this object, especially with NLTK's trees? Here's some help: https://stackoverflow.com/questions/26210567/get-entities-from-nltk-tree-result

In [3]:
foo_trees = parser.raw_parse("It was a bright cold day in April and the clocks were striking thirteen.")

In [4]:
type(foo_trees)

list_iterator

In [5]:
for tree in foo_trees:
    tree.leaves()

['It',
 'was',
 'a',
 'bright',
 'cold',
 'day',
 'in',
 'April',
 'and',
 'the',
 'clocks',
 'were',
 'striking',
 'thirteen',
 '.']

In [6]:
stimuli_sheet = pd.read_excel('Stimuli.xlsx')
print(stimuli_sheet)

    ORDER                                               Item
0       9  In the right hands, location data can be a for...
1      19  Paying for college seems out of reach for many...
2      20  Very few issues can bring together lawmakers o...
3      49  Koalas have been running into hard times. They...
4      51  Some recent reports have suggested that women ...
..    ...                                                ...
58    206  Earth is warming, and we know why. Light is re...
59    208  Nobody ever really tells you how to breathe. Y...
60    211  The immense wall of crumbling, whitewashed sto...
61    212  You never know where you will find love. Even ...
62    215  Imagine an animal that looks like a dinosaur, ...

[63 rows x 2 columns]


In [7]:
sents = stimuli_sheet["Item"]

In [8]:
sents[:5]
sents[0]

0    In the right hands, location data can be a for...
1    Paying for college seems out of reach for many...
2    Very few issues can bring together lawmakers o...
3    Koalas have been running into hard times. They...
4    Some recent reports have suggested that women ...
Name: Item, dtype: object

'In the right hands, location data can be a force for good. It can provide publishers and app developers with advertising revenue so consumers can access free services and information.'

In [9]:
sent1_trees = []
sent1_productions = []
sent2_trees = []
sent2_productions = []
sentCount=1
for entry in sents:
    sent1, sent2 = nltk.sent_tokenize(entry)
    trees1 = parser.raw_parse(sent1)
    trees2 = parser.raw_parse(sent2)
    for tree in trees1:
        print("The first tree for sentence #",sentCount," is:")
        tree.pretty_print()
        sent1_productions.append(tree.productions())
        sent1_trees.append(tree)
    for tree in trees2:
        print("The second tree for sentence #",sentCount," is:")
        tree.pretty_print()
        sent2_productions.append(tree.productions())
        sent2_trees.append(tree)
    sentCount+=1

The first tree for sentence # 1  is:
                                         ROOT                                           
                                          |                                              
                                          S                                             
          ________________________________|___________________________________________   
         |               |            |                VP                             | 
         |               |            |         _______|________                      |  
         |               |            |        |                VP                    | 
         |               |            |        |    ____________|____                 |  
         |               |            |        |   |                 NP               | 
         |               |            |        |   |        _________|_______         |  
         PP              |            |        |   |       |        

The first tree for sentence # 4  is:
                   ROOT                             
                    |                                
                    S                               
   _________________|_____________________________   
  |                 VP                            | 
  |      ___________|_____                        |  
  |     |                 VP                      | 
  |     |     ____________|____                   |  
  |     |    |                 VP                 | 
  |     |    |       __________|___               |  
  |     |    |      |              PP             | 
  |     |    |      |      ________|____          |  
  NP    |    |      |     |             NP        | 
  |     |    |      |     |         ____|____     |  
 NNPS  VBP  VBN    VBG    IN       JJ       NNS   . 
  |     |    |      |     |        |         |    |  
Koalas have been running into     hard     times  . 

The second tree for sentence # 4  is:
               

The first tree for sentence # 6  is:
                          ROOT                       
                           |                          
                           S                         
             ______________|_______________________   
            |                   VP                 | 
            |           ________|___               |  
            NP         |            NP             | 
     _______|____      |    ________|______        |  
    JJ           NN   VBZ  DT       JJ     NN      . 
    |            |     |   |        |      |       |  
Pancreatic     cancer has  a       bad reputation  . 

The second tree for sentence # 6  is:
                                                             ROOT                                                                      
                                                              |                                                                         
                                                    


The second tree for sentence # 8  is:
                                    ROOT                                                 
                                     |                                                    
                                     S                                                   
  ___________________________________|_________________________________________________   
 |        |                      VP                                                    | 
 |        |            __________|____________                                         |  
 |        |           |                      SBAR                                      | 
 |        |           |                       |                                        |  
 |        |           |                       S                                        | 
 |        |           |       ________________|_________                               |  
 |        |           |      |                          

The first tree for sentence # 11  is:
                                                                 ROOT                                                                      
                                                                  |                                                                         
                                                                  S                                                                        
                                    ______________________________|______________________________________________________________________   
                                   S                                                                               |     |        |      | 
                 __________________|__________                                                                     |     |        |      |  
                |                             VP                                                                   |   

...

The first tree for sentence # 54  is:
                                              ROOT                                                
                                               |                                                   
                                               S                                                  
                         ______________________|________________________________________________   
                        |                               VP                                      | 
                        |                   ____________|_____                                  |  
                        |                  |   |              NP                                | 
                        |                  |   |         _____|_____________________            |  
                        NP                 |   |        |                          SBAR         | 
         _______________|___               |   |        |          


The first tree for sentence # 56  is:
                                             ROOT                                        
                                              |                                           
                                              S                                          
           ___________________________________|________________________________________   
          PP                                  |        |                |              | 
  ________|________                           |        |                |              |  
 |                 NP                         |        |                |              | 
 |         ________|____________              |        |                |              |  
 |        |                     PP            |        |                VP             | 
 |        |                  ___|______       |        |             ___|___           |  
 |        NP                |          NP     |        N

The first tree for sentence # 58  is:
                                             ROOT                                       
                                              |                                          
                                              S                                         
     _________________________________________|_______________________________________   
    |                                         VP                                      | 
    |        _________________________________|___________                            |  
    |       |      |           |                          PP                          | 
    |       |      |           |               ___________|____                       |  
    |       |      |           |              |                S                      | 
    |       |      |           |              |                |                      |  
    |       |      |           |              |                VP  

The first tree for sentence # 60  is:
                         ROOT                            
                          |                               
                          S                              
   _______________________|____________________________   
  |     |                 VP                           | 
  |     |      ___________|___________                 |  
  |     |     |      |    |          SBAR              | 
  |     |     |      |    |      _____|____            |  
  |     |     |      |    |     |          S           | 
  |     |     |      |    |     |          |           |  
  |     |     |      |    |     |          VP          | 
  |     |     |      |    |     |      ____|_____      |  
  NP   ADVP  ADVP    |    NP  WHADVP  |          VP    | 
  |     |     |      |    |     |     |          |     |  
  NN    RB    RB    VBZ  PRP   WRB    TO         VB    . 
  |     |     |      |    |     |     |          |     |  
Nobody ever really tells y

The first tree for sentence # 63  is:
                                                            ROOT                                                  
                                                             |                                                     
                                                             S                                                    
                         ____________________________________|__________________________________________________   
                        S                                    |    |                     |                       | 
                        |                                    |    |                     |                       |  
                        VP                                   |    |                     |                       | 
    ____________________|_____                               |    |                     |                       |  
   |                          NP      

In [10]:
sent1_productions[0]

[ROOT -> S,
 S -> PP , NP VP .,
 PP -> IN NP,
 IN -> 'In',
 NP -> DT JJ NNS,
 DT -> 'the',
 JJ -> 'right',
 NNS -> 'hands',
 , -> ',',
 NP -> NN NNS,
 NN -> 'location',
 NNS -> 'data',
 VP -> MD VP,
 MD -> 'can',
 VP -> VB NP,
 VB -> 'be',
 NP -> NP PP,
 NP -> DT NN,
 DT -> 'a',
 NN -> 'force',
 PP -> IN ADJP,
 IN -> 'for',
 ADJP -> JJ,
 JJ -> 'good',
 . -> '.']

In [11]:
all_prods = []
for lists in sent1_productions:
    for rules in lists:
        all_prods.append(rules)
for lists in sent2_productions:
    for rules in lists:
        all_prods.append(rules)
len(all_prods)
set_prods = set(all_prods)    
len(set_prods)
set_prods

3728

1246

{NNS -> 'bodies',
 NN -> 'majority',
 S -> S , IN S .,
 NNS -> 'guns',
 VP -> VBZ S , S,
 S -> S VP .,
 VP -> VP CC VP,
 NN -> 'protection',
 VBP -> 'thrive',
 VB -> 'bring',
 RB -> 'ago',
 NP -> VBG JJ NNS,
 VBN -> 'stored',
 NP -> NP , NP CC NP,
 VB -> 'imagine',
 PP -> IN NP,
 NN -> 'television',
 VB -> 'destroy',
 RB -> 'inevitably',
 JJ -> 'subject',
 JJ -> 'sudden',
 JJ -> 'effective',
 VBP -> 'haul',
 NN -> 'center',
 JJ -> 'broad',
 VBG -> 'worrying',
 IN -> 'without',
 SBAR -> WHADVP S,
 JJ -> 'deep',
 VBG -> 'dying',
 NN -> 'purge',
 VB -> 'fuel',
 NN -> 'scene',
 S -> CC ADVP NP VP .,
 NP -> NNS,
 VBP -> 'show',
 WHNP -> WDT,
 NP -> DT JJ NN,
 ADJP -> JJR RP S,
 NNS -> 'Years',
 JJ -> 'lakeside',
 NN -> 'nutrition',
 ADVP -> RB PP,
 NN -> 'amount',
 NN -> 'warmth',
 ADJP -> RB JJR,
 VB -> 'consider',
 JJ -> 'experienced',
 RB -> 'apart',
 NN -> 'Toothpaste',
 NNS -> 'experts',
 VBN -> 'blessed',
 VP -> VBN S,
 CC -> 'and',
 VB -> 'stretch',
 NNP -> 'Income',
 VB -> 'grieve',

In [12]:
all_prods[:10]

[ROOT -> S,
 S -> PP , NP VP .,
 PP -> IN NP,
 IN -> 'In',
 NP -> DT JJ NNS,
 DT -> 'the',
 JJ -> 'right',
 NNS -> 'hands',
 , -> ',',
 NP -> NN NNS]

In [13]:
leaves = []
interior = []
count=0
for rule in all_prods:
    test = str(rule) #can't do .contains()/in for a productions datatype, cool
    if "'" in test:
        leaves.append(rule)
    else:
        interior.append(rule)
    left, right = test.split("->")
    count+=1
len(leaves)
len(interior)
count
leaves[:10]
interior[:10]

1932

1796

3728

[IN -> 'In',
 DT -> 'the',
 JJ -> 'right',
 NNS -> 'hands',
 , -> ',',
 NN -> 'location',
 NNS -> 'data',
 MD -> 'can',
 VB -> 'be',
 DT -> 'a']

[ROOT -> S,
 S -> PP , NP VP .,
 PP -> IN NP,
 NP -> DT JJ NNS,
 NP -> NN NNS,
 VP -> MD VP,
 VP -> VB NP,
 NP -> NP PP,
 NP -> DT NN,
 PP -> IN ADJP]

In [14]:
leftRules = []
rightRules = []
for rule in set_prods:
    text = str(rule)
    left, right = text.split(" -> ")
    leftRules.append(left)
    rightRules.append(right)

In [15]:
len(leftRules)
len(rightRules) #these aren't that useful?
cats = set(leftRules)
len(cats)
cats

1246

1246

48

{',',
 '.',
 'ADJP',
 'ADVP',
 'CC',
 'CD',
 'DT',
 'EX',
 'FRAG',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'NP',
 'NP-TMP',
 'PDT',
 'PP',
 'PRP',
 'PRP$',
 'PRT',
 'QP',
 'RB',
 'RBR',
 'RBS',
 'ROOT',
 'RP',
 'S',
 'SBAR',
 'SINV',
 'TO',
 'UCP',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'VP',
 'WDT',
 'WHADVP',
 'WHNP',
 'WP',
 'WRB'}

In [16]:
#file = open('cats.pkl','wb')
#pickle.dump(cats, file)

## Validating Results

In [17]:
file = open('parse.output.txt')
text = file.readlines()
file.close()
parseOutput = []
for line in text:
    if line.startswith("1"):
        parseOutput.append(line)

In [18]:
help(sent1_trees[0])

Help on Tree in module nltk.tree object:

class Tree(builtins.list)
 |  Tree(node, children=None)
 |  
 |  A Tree represents a hierarchical grouping of leaves and subtrees.
 |  For example, each constituent in a syntax tree is represented by a single Tree.
 |  
 |  A tree's children are encoded as a list of leaves and subtrees,
 |  where a leaf is a basic (non-tree) value; and a subtree is a
 |  nested Tree.
 |  
 |      >>> from nltk.tree import Tree
 |      >>> print(Tree(1, [2, Tree(3, [4]), 5]))
 |      (1 2 (3 4) 5)
 |      >>> vp = Tree('VP', [Tree('V', ['saw']),
 |      ...                  Tree('NP', ['him'])])
 |      >>> s = Tree('S', [Tree('NP', ['I']), vp])
 |      >>> print(s)
 |      (S (NP I) (VP (V saw) (NP him)))
 |      >>> print(s[1])
 |      (VP (V saw) (NP him))
 |      >>> print(s[1,1])
 |      (NP him)
 |      >>> t = Tree.fromstring("(S (NP I) (VP (V saw) (NP him)))")
 |      >>> s == t
 |      True
 |      >>> t[1][1].set_label('X')
 |      >>> t[1][1].label()


In [24]:
len(sent1_trees + sent2_trees)
len(parseOutput)
len(sent1_trees)
len(sent2_trees)

i=0
c=0
while(i<126):
    print("Trees for Sentence Pair", c)
    print()
    print("Sentence One Trees: ")
    sent1_trees[c].pprint()
    print(parseOutput[i])
    i+=1
    print("Sentence Two Trees: ")
    sent2_trees[c].pprint()
    print(parseOutput[i])
    i+=1
    c+=1
    print()

126

126

63

63

Trees for Sentence Pair 0

Sentence One Trees: 
(ROOT
  (S
    (PP (IN In) (NP (DT the) (JJ right) (NNS hands)))
    (, ,)
    (NP (NN location) (NNS data))
    (VP
      (MD can)
      (VP
        (VB be)
        (NP (NP (DT a) (NN force)) (PP (IN for) (ADJP (JJ good))))))
    (. .)))
1   1    0.5249   75.9911   51.6965   24.2946	(TOP (S (PP (IN In) (NP (DT the) (JJ right) (NNS hands))) (, ,) (NP (NN location) (NNS data)) (VP (MD can) (VP (VB be) (NP (NP (DT a) (NN force)) (PP (IN for) (NP (JJ good)))))) (. .)))

Sentence Two Trees: 
(ROOT
  (S
    (NP (PRP It))
    (VP
      (MD can)
      (VP
        (VB provide)
        (NP (NNS publishers) (CC and) (NN app) (NNS developers))
        (PP (IN with) (NP (NN advertising) (NN revenue)))
        (SBAR
          (IN so)
          (S
            (NP (NNS consumers))
            (VP
              (MD can)
              (VP
                (VB access)
                (NP
                  (JJ free)
                  (NNS services)
         